# v9fs performance studies

First thing we need to do is start a qemu with a kernel.  The versions of both of these matter if what we are doing is qemu-virtio as transport (and probably also have an impact on qemu network performance)

TODO:
- [ ] TASK: add build configuraton for [qemu | kernel | u-root | etc]

# Setup Test Environment

The following command starts qemu in the background.  You can optionally set the QEMU command line and KERNEL to use as environment variables.
You must be setup for kvm and the user must have permission to start kvm instances without sudo (usually by being in group kvm).

Okay, so this all works ... but we don't have any nice way to cleanup, so we'll have to work that out at some point.

In [1]:
QEMU = "../qemu/build/qemu-system-x86_64"
KERNEL = "../build/arch/x86_64/boot/bzImage"


In [10]:
!sg kvm -c '{QEMU} -kernel \
    {KERNEL} \
	-cpu  max \
    -s   \
    -enable-kvm \
    -m 8192m \
    -machine q35  \
    -initrd /tmp/initramfs.linux_amd64.cpio \
    -object rng-random,filename=/dev/urandom,id=rng0 \
    -device virtio-rng-pci,rng=rng0 \
    -device vhost-vsock-pci,guest-cid=42 \
    -device virtio-net-pci,netdev=n1 \
    -netdev user,id=n1,hostfwd=tcp:127.0.0.1:17010-:17010,net=192.168.1.0/24,host=192.168.1.1 \
    -debugcon file:debug.log -global isa-debugcon.iobase=0x402 \
    -fsdev local,security_model=passthrough,id=fsdev0,path=/tmp \
    -device virtio-9p-pci,id=fs0,fsdev=fsdev0,mount_tag=hostshare \
    -daemonize'

qemu-system-x86_64: -netdev user,id=n1,hostfwd=tcp:127.0.0.1:17010-:17010,net=192.168.1.0/24,host=192.168.1.1: Could not set up host forwarding rule 'tcp:127.0.0.1:17010-:17010'


# Run a test

Okay, so now we need some simple tests to run in this environment.  We'll do this all over cpu.

In [14]:
test1 = %timeit -n1 -r5 !cpu -fstab fstab localhost dd if=test of=/mnt/root/test bs=4096 count=4000 2>&1 | tail -1 | awk '{print $(NF-1),$(NF)}'

11.5 MB/s
11.3 MB/s
11.6 MB/s
12.0 MB/s
10.5 MB/s
1.71 s ± 69.8 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [39]:
test_list = []

for i in range(5):
    test2 = !cpu -fstab fstab localhost dd if=test of=/mnt/root/test bs=4096 2>&1 | tail -1 | awk '{print $(NF-1),$(NF)}'
    test3 = test2[0].split()
    if(test3[1]=="MB/s"):
        test_list.append(float(test3[0]))


In [49]:
time_list = []
for i in range(2):
    result = !cpu -fstab fstab localhost /usr/bin/time -f "%e" dd if=test of=/mnt/root/test bs=4096 status=none 2> err
    time_list.append(float(result[0]))

In [54]:
print(str(1000 / time_list[0]))
print(str(1000 / time_list[1]))

11.053387863380125
11.230907457322552


In [40]:
mean = sum(test_list) / len(test_list)
variance = sum([((x - mean) ** 2) for x in test_list]) / len(test_list)
res = variance ** 0.5
print("Mean of sample is : " + str(mean))
print("Min of sample is : " + str(min(test_list)))
print("Max of sample is : " + str(max(test_list)))
print("Variance of sample is : " + str(variance))
print("Standard deviation of sample is : " + str(res))




Mean of sample is : 11.9
Min of sample is : 11.5
Max of sample is : 12.1
Variance of sample is : 0.06399999999999997
Standard deviation of sample is : 0.2529822128134703


In [42]:
def test(vt):
    print("Hello, " + vt)

test("Squidboy")

Hello, Squidboy


In [43]:
test("World")

Hello, World


In [8]:
import pandas as pd

cf = pd.DataFrame({
                    "a" : ["thundercleese", 6.0],
                    "b" : ["thundercleese", 5.1],
                    "c" : ["rpi", 5.11]
                })

df = pd.DataFrame({"a" : [("thundercleese", 6.0),("thundercleese", 5.11),("rpi", 5.11)],
                   "b" : [7,8,9],
                   "c" : [10,11,12]},
                   index = [ 1, 2, 3])

ef = pd.read_csv("test.csv")

In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import subprocess

#gcc_proc = subprocess.run(['gcc', '-v'], check=True, stderr=PIPE ).stderr

subprocess.check_output("gcc -v 2>&1 | tail -1", shell=True).split()[2]

b'12.2.0'

In [19]:
subprocess.check_output("go version 2>&1", shell=True).split()[2]

b'go1.19.2'

In [10]:
print(gcc_vers)

b''


In [26]:
import time
time.time()

1669223138.1844015

In [27]:
subprocess.check_output("wc -l configs.csv", shell=True).decode('utf-8').split()[0]

'3'

In [35]:
foo = subprocess.check_output("cpu localhost echo hello", shell=True).decode('utf-8')

2022/11/23 18:25:19 Can not get winsize: inappropriate ioctl for device; assuming 80x40 and non-interactive


KeyboardInterrupt: 

In [5]:
import bench

configs = [ 'alpha', 'beta', 'delta']
blocksizes = [ 512, 1024, 4096, 8192, 32768, 65536, 131072 ]
iterations = 5

bench.run( configs, blocksizes, iterations, "Squidboy")


Squidboy
config: alpha blocksize: 512 iteration: 0
config: alpha blocksize: 512 iteration: 1
config: alpha blocksize: 512 iteration: 2
config: alpha blocksize: 512 iteration: 3
config: alpha blocksize: 512 iteration: 4
config: alpha blocksize: 1024 iteration: 0
config: alpha blocksize: 1024 iteration: 1
config: alpha blocksize: 1024 iteration: 2
config: alpha blocksize: 1024 iteration: 3
config: alpha blocksize: 1024 iteration: 4
config: alpha blocksize: 4096 iteration: 0
config: alpha blocksize: 4096 iteration: 1
config: alpha blocksize: 4096 iteration: 2
config: alpha blocksize: 4096 iteration: 3
config: alpha blocksize: 4096 iteration: 4
config: alpha blocksize: 8192 iteration: 0
config: alpha blocksize: 8192 iteration: 1
config: alpha blocksize: 8192 iteration: 2
config: alpha blocksize: 8192 iteration: 3
config: alpha blocksize: 8192 iteration: 4
config: alpha blocksize: 32768 iteration: 0
config: alpha blocksize: 32768 iteration: 1
config: alpha blocksize: 32768 iteration: 2
conf

In [2]:
from tqdm import trange, tqdm
from time import sleep

for i in trange(3, desc='1st loop'):
    for j in tqdm(range(100), desc='2nd loop'):
        sleep(0.01)

1st loop: 100%|██████████| 3/3 [00:03<00:00,  1.03s/it]


So I could go with that, and that seems useful. Need to think about